In [2]:
from sklearn.preprocessing import LabelBinarizer,MinMaxScaler,LabelEncoder,OneHotEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import concatenate
import matplotlib.pyplot as plt
import datetime
import pandas as pd
import numpy as np
import argparse
import locale
import glob
import cv2
import os

In [3]:
HOY = pd.to_datetime(datetime.date(2021,6,30))

In [4]:
df = pd.read_csv('train_original.csv')

In [5]:
encoder=OneHotEncoder()

In [6]:
df['Insurance_company'].value_counts()

B     148
A     132
BQ    131
C     131
DA    128
O     126
BB    124
AA    123
BC    121
AC    119
RE    116
Name: Insurance_company, dtype: int64

In [7]:
df.head()

,Image_path,Insurance_company,Cost_of_vehicle,Min_coverage,Expiry_date,Max_coverage,Condition,Amount
0,img_4513976.jpg,BQ,41500.0,1037.5,2026-12-03,36142.68,0,0.0
1,img_7764995.jpg,BQ,50700.0,1267.5,2025-07-10,12753.00,1,6194.0
2,img_451308.jpg,A,49500.0,1237.5,2022-08-11,43102.68,0,0.0
3,img_7768372.jpg,A,33500.0,837.5,2022-08-02,8453.00,1,7699.0
4,img_7765274.jpg,AC,27600.0,690.0,2026-05-01,6978.00,1,8849.0


In [8]:
encoder.fit(df[['Insurance_company']])

OneHotEncoder()

In [9]:
def preprocesa_atributos(input_path):
    df=pd.read_csv(input_path)
    df.dropna(inplace=True)
    df = df.drop(columns=['Condition'])
    df = df[(df['Amount']<=df['Cost_of_vehicle'])]
    
    df['Expiry_date']=pd.to_datetime(df['Expiry_date'])
    df['dias_pendientes_cobertura']=(df['Expiry_date']-HOY).dt.days.astype(int)
    df['relative_amount'] = df['Amount']/df['Cost_of_vehicle']
    df['relative_max_cov'] = df['Max_coverage']/df['Cost_of_vehicle']
    
    
    df = df.drop(columns=['Min_coverage','Expiry_date','Max_coverage','Amount'])
    
    scaler = MinMaxScaler()
    encoder = OneHotEncoder(handle_unknown='ignore')
    
    X = df.drop(columns=['relative_amount'])
    y = df['relative_amount']
    
    X_train,X_val,y_train,y_val = train_test_split(X, y, test_size=0.25, random_state=777)
    
    
    X_train_num = X_train[['Cost_of_vehicle','dias_pendientes_cobertura']]
    X_val_num = X_val[['Cost_of_vehicle','dias_pendientes_cobertura']]
    
    X_train_num_scaled = pd.DataFrame(scaler.fit_transform(X_train_num), 
                                      columns=X_train_num.columns,
                                      index=X_train.index)
    
    X_val_num_scaled = pd.DataFrame(scaler.transform(X_val_num), 
                                      columns=X_val_num.columns,
                                       index=X_val.index)
    
    lb = LabelBinarizer()
    
    
    X_train_encoded = pd.DataFrame(lb.fit_transform(X_train['Insurance_company']),
                                    columns=lb.classes_,
                                   index=X_train.index)
    
    X_val_encoded = pd.DataFrame(lb.transform(X_val['Insurance_company']),
                                    columns=lb.classes_,
                                   index=X_val.index)
    
    
    X_train_feat = pd.concat([X_train['relative_max_cov'],X_train_num_scaled,X_train_encoded],axis=1)
    X_val_feat = pd.concat([X_val['relative_max_cov'],X_val_num_scaled,X_val_encoded],axis=1)
    
    X_train_im = X_train['Image_path']
    X_val_im = X_val['Image_path']
    
    
    return X_train_feat,X_train_im, X_val_feat,X_val_im,y_train,y_val
    

In [10]:
X_train_feat,X_train_im, X_val_feat,X_val_im,y_train,y_val = preprocesa_atributos('train_original.csv')

Si quisiera inspeccionar imagenes

In [36]:
'''
i=1 

for indice in X_train_im.index:

    img = cv2.imread('trainImages/'+X_train_im[indice])
    trans_img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    plt.imshow(trans_img)
    plt.title('foto '+str(i)+':'+str(y_train[indice]))
    plt.show()
    i+=1
'''

"\ni=1 \n\nfor indice in X_train_im.index:\n\n    img = cv2.imread('trainImages/'+X_train_im[indice])\n    trans_img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)\n\n    plt.imshow(trans_img)\n    plt.title('foto '+str(i)+':'+str(y_train[indice]))\n    plt.show()\n    i+=1\n"

In [64]:
def carga_imagenes(serie_fotos, inputPath):
    
    nuevas_imagenes = []
    for foto in serie_fotos:
        imagen = cv2.imread(inputPath+'/'+foto)
        imagen = cv2.resize(imagen, (128, 128))
        nuevas_imagenes.append(imagen)
    return np.array(nuevas_imagenes)/255.

In [83]:
type(carga_imagenes(X_val_im,'trainImages'))

numpy.ndarray

In [55]:
X_val_feat.shape

(325, 14)

In [56]:
def create_mlp(dim, regress=False):
    # define our MLP network
    model = Sequential()
    model.add(Dense(8, input_dim=dim, activation="relu"))
    model.add(Dense(4, activation="relu"))

    # check to see if the regression node should be added
    if regress:
        model.add(Dense(1, activation="linear"))

    # return our model
    return model

In [60]:
def create_cnn(width, height, depth, filters=(16, 32, 64), regress=False):
# initialize the input shape and channel dimension, assuming
# TensorFlow/channels-last ordering
    inputShape = (height, width, depth)
    chanDim = -1

    # define the model input
    inputs = Input(shape=inputShape)

    # loop over the number of filters
    for (i, f) in enumerate(filters):
    # if this is the first CONV layer then set the input
    # appropriately
        if i == 0:
            x = inputs

    # CONV => RELU => BN => POOL
    x = Conv2D(f, (3, 3), padding="same")(x)
    x = Activation("relu")(x)
    x = BatchNormalization(axis=chanDim)(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)

    # flatten the volume, then FC => RELU => BN => DROPOUT
    x = Flatten()(x)
    x = Dense(16)(x)
    x = Activation("relu")(x)
    x = BatchNormalization(axis=chanDim)(x)
    x = Dropout(0.5)(x)

    # apply another FC layer, this one to match the number of nodes
    # coming out of the MLP
    x = Dense(4)(x)
    x = Activation("relu")(x)

    # check to see if the regression node should be added
    if regress:
        x = Dense(1, activation="linear")(x)

    # construct the CNN
    model = Model(inputs, x)

    # return the CNN
    return model

In [84]:
imagenes_train = carga_imagenes(X_train_im,'trainImages')
imagenes_val = carga_imagenes(X_val_im,'trainImages')

In [85]:
mlp = create_mlp(X_train_feat.shape[1], regress=False)

In [86]:
cnn = create_cnn(128, 128, 3, regress=False)

In [87]:
combinedInput = concatenate([mlp.output, cnn.output])

In [88]:
x = Dense(4, activation="relu")(combinedInput)
x = Dense(1, activation="linear")(x)

In [89]:
model = Model(inputs=[mlp.input, cnn.input], outputs=x)

In [ ]:
opt = Adam(lr=1e-3, decay=1e-3 / 200)
model.compile(loss="mean_absolute_percentage_error", optimizer=opt)

# train the model
print("[INFO] training model...")
model.fit(
    x=[X_train_feat.values, imagenes_train], y=y_train.values,
    validation_data=([X_val_feat.values, imagenes_val], y_val.values),
    epochs=200, batch_size=8)

# make predictions on the testing data
#print("[INFO] predicting house prices...")
#preds = model.predict([testAttrX, testImagesX])

/Users/ggoni/opt/anaconda3/envs/dl/lib/python3.9/site-packages/keras/optimizer_v2/optimizer_v2.py:355: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(


[INFO] training model...


2021-10-08 01:02:26.054627: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/200
122/122 [==============================] - 16s 123ms/step - loss: 31520784.0000 - val_loss: 2529709.7500
Epoch 2/200
122/122 [==============================] - 15s 120ms/step - loss: 16405226.0000 - val_loss: 5406155.0000
Epoch 3/200
122/122 [==============================] - 14s 118ms/step - loss: 14240994.0000 - val_loss: 4377897.0000
Epoch 4/200
122/122 [==============================] - 15s 119ms/step - loss: 8712942.0000 - val_loss: 2850682.5000
Epoch 5/200
122/122 [==============================] - 16s 131ms/step - loss: 5322680.0000 - val_loss: 2421362.2500
Epoch 6/200
122/122 [==============================] - 15s 122ms/step - loss: 6978489.5000 - val_loss: 263861.7188
Epoch 7/200
122/122 [==============================] - 16s 134ms/step - loss: 3124890.5000 - val_loss: 1370651.3750
Epoch 8/200
122/122 [==============================] - 16s 129ms/step - loss: 2225047.0000 - val_loss: 164923.1562
Epoch 9/200
122/122 [==============================] - 16s 127ms/step -

In [82]:
imagenes_train

(996     img_4538327.jpg
 1132     img_451799.jpg
 720     img_4515734.jpg
 209     img_4518363.jpg
 438     img_4634857.jpg
              ...       
 695     img_7768762.jpg
 83      img_4538608.jpg
 1013    img_4514629.jpg
 886     img_7766226.jpg
 119     img_4638780.jpg
 Name: Image_path, Length: 975, dtype: object,
 'trainImages')

In [77]:
y_train.values

array([0.0080863 , 0.        , 0.00879699, 0.16507463, 0.08255528,
       0.        , 0.08794872, 0.        , 0.0132287 , 0.05426104,
       0.1478211 , 0.20750988, 0.23074074, 0.13422983, 0.14958084,
       0.16285714, 0.07604651, 0.17280936, 0.07137014, 0.10022774,
       0.24662602, 0.21387097, 0.02862832, 0.34282759, 0.08180467,
       0.        , 0.12256522, 0.0129653 , 0.166691  , 0.12137555,
       0.10124294, 0.19170799, 0.10311213, 0.02390313, 0.25870968,
       0.17178922, 0.11      , 0.01464135, 0.19801843, 0.14967914,
       0.25170455, 0.15461126, 0.14420613, 0.01171488, 0.06153846,
       0.02138318, 0.11220561, 0.02141876, 0.2347482 , 0.02484848,
       0.00982759, 0.17596354, 0.08935733, 0.06567982, 0.06058824,
       0.17432373, 0.15747541, 0.28468085, 0.10927492, 0.1763986 ,
       0.1865252 , 0.11592476, 0.22330396, 0.12216963, 0.10065217,
       0.16134804, 0.09046371, 0.06621032, 0.08930288, 0.18332203,
       0.00876448, 0.1114717 , 0.16874652, 0.20843949, 0.03800